In [ ]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
def plot_class_distribution(water_pixels, total_pixels, out_path="class_distribution.png"):
    non_water_pixels = total_pixels - water_pixels

    labels = ["Non-Water", "Water"]
    sizes = [non_water_pixels, water_pixels]

    plt.figure(figsize=(6,6))
    plt.pie(sizes, labels=labels, autopct="%.2f%%", startangle=90, colors=["skyblue", "dodgerblue"])
    plt.title("Water vs Non-Water Pixel Distribution")
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

In [ ]:
def compute_flood_stats(root):
    root = Path(root)
    total_pixels = 0
    flood_pixels = 0
    scene_ratios = {}

    for scene_dir in root.iterdir():
        print("Processing scene:", scene_dir.name)
        mask_dir = scene_dir / "tiles" / "label"
        scene_total = 0
        scene_flood = 0

        for mask_path in mask_dir.glob("*.png"):
            mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
            mask_bin = (mask > 127).astype(np.uint8)

            scene_total += mask_bin.size
            scene_flood += mask_bin.sum()

        if scene_total > 0:
            scene_ratios[scene_dir.name] = scene_flood / scene_total

        total_pixels += scene_total
        flood_pixels += scene_flood

    flood_ratio = flood_pixels / total_pixels
    return flood_ratio, flood_pixels, total_pixels, scene_ratios

In [ ]:
def plot_water_ratio_per_scene(scene_ratios, out_path="water_ratio_per_scene.png"):
    scenes = list(scene_ratios.keys())
    ratios = list(scene_ratios.values())

    plt.figure(figsize=(12,6))
    plt.bar(scenes, ratios, color="steelblue")
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Water Ratio")
    plt.title("Water Pixel Ratio Per Scene")
    plt.tight_layout()
    plt.savefig(out_path)
    plt.close()

In [ ]:
OUT_ROOT = Path("")
ratio, flood_px, total_px, scene_ratios = compute_flood_stats(OUT_ROOT)

print("Overall Water Ratio:", ratio)
print("Water Pixels:", flood_px)
print("Total Pixels:", total_px)

plot_class_distribution(flood_px, total_px, "overall_class_distribution.png")
plot_water_ratio_per_scene(scene_ratios, "scene_water_ratios.png")